[View in Colaboratory](https://colab.research.google.com/github/udithv/NYC-TAXI-kaggle/blob/master/NYC_taxi.ipynb)

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#data dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
import seaborn as sns

from sklearn.model_selection import train_test_split
import xgboost as xgb


%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False



In [0]:
#authenticate

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [5]:
#test - 1o8uTatNbW8zbsjthc5nEWu2i74lRPwEq
#train - 14JDkgbC40awSogWBpsC1cM8Q2l8D7sIF


train_downloaded = drive.CreateFile({'id': '14JDkgbC40awSogWBpsC1cM8Q2l8D7sIF'})
train_downloaded.GetContentFile('train.csv')
test_downloaded = drive.CreateFile({'id': '1o8uTatNbW8zbsjthc5nEWu2i74lRPwEq'})
test_downloaded.GetContentFile('test.csv')


df_train = pd.read_csv('train.csv')
df_train

id  vendor_id      pickup_datetime     dropoff_datetime  \
0        id2875421          2  2016-03-14 17:24:55  2016-03-14 17:32:30   
1        id2377394          1  2016-06-12 00:43:35  2016-06-12 00:54:38   
2        id3858529          2  2016-01-19 11:35:24  2016-01-19 12:10:48   
3        id3504673          2  2016-04-06 19:32:31  2016-04-06 19:39:40   
4        id2181028          2  2016-03-26 13:30:55  2016-03-26 13:38:10   
5        id0801584          2  2016-01-30 22:01:40  2016-01-30 22:09:03   
6        id1813257          1  2016-06-17 22:34:59  2016-06-17 22:40:40   
7        id1324603          2  2016-05-21 07:54:58  2016-05-21 08:20:49   
8        id1301050          1  2016-05-27 23:12:23  2016-05-27 23:16:38   
9        id0012891          2  2016-03-10 21:45:01  2016-03-10 22:05:26   
10       id1436371          2  2016-05-10 22:08:41  2016-05-10 22:29:55   
11       id1299289          2  2016-05-15 11:16:11  2016-05-15 11:34:59   
12       id1187965          2  2016-02-19 09:52:46  2016-02-19 10:11:20   
13       id0799785          2  2016-06-01 20:58:29  2016-06-01 21:02:49   
14       id2900608          2  2016-05-27 00:43:36  2016-05-27 01:07:10   
15       id3319787          1  2016-05-16 15:29:02  2016-05-16 15:32:33   
16       id3379579          2  2016-04-11 17:29:50  2016-04-11 18:08:26   
17       id1154431          1  2016-04-14 08:48:26  2016-04-14 09:00:37   
18       id3552682          1  2016-06-27 09:55:13  2016-06-27 10:17:10   
19       id3390316          2  2016-06-05 13:47:23  2016-06-05 13:51:34   
20       id2070428          1  2016-02-28 02:23:02  2016-02-28 02:31:08   
21       id0809232          2  2016-04-01 12:12:25  2016-04-01 12:23:17   
22       id2352683          1  2016-04-09 03:34:27  2016-04-09 03:41:30   
23       id1603037          1  2016-06-25 10:36:26  2016-06-25 10:55:49   
24       id3321406          2  2016-06-03 08:15:05  2016-06-03 08:56:30   
25       id0129640          2  2016-02-14 13:27:56  2016-02-14 13:49:19   
26       id3587298          1  2016-02-27 21:56:01  2016-02-27 22:14:51   
27       id2104175          1  2016-06-20 23:07:16  2016-06-20 23:18:50   
28       id3973319          2  2016-06-13 21:57:27  2016-06-13 22:12:19   
29       id1410897          1  2016-03-23 14:10:39  2016-03-23 14:49:30   
...            ...        ...                  ...                  ...   
1458614  id2061444          2  2016-02-08 17:16:07  2016-02-08 17:21:45   
1458615  id3182230          1  2016-02-05 17:57:08  2016-02-05 18:11:25   
1458616  id2822294          1  2016-04-22 17:21:14  2016-04-22 17:29:22   
1458617  id0820021          2  2016-04-15 08:31:20  2016-04-15 08:34:48   
1458618  id1046767          2  2016-04-17 01:46:48  2016-04-17 01:52:55   
1458619  id1083860          2  2016-04-23 12:14:15  2016-04-23 12:26:03   
1458620  id0694577          2  2016-04-28 20:51:03  2016-04-28 21:10:25   
1458621  id3267199          2  2016-05-09 14:33:30  2016-05-09 15:12:45   
1458622  id0125435          2  2016-02-19 18:26:52  2016-02-19 18:36:04   
1458623  id3369208          1  2016-01-18 20:35:30  2016-01-18 20:44:44   
1458624  id3482902          1  2016-03-01 07:21:04  2016-03-01 07:23:36   
1458625  id3730733          2  2016-01-25 17:21:15  2016-01-25 17:54:37   
1458626  id0155863          2  2016-01-17 17:21:11  2016-01-17 17:25:15   
1458627  id0439281          2  2016-06-23 10:10:28  2016-06-23 10:25:08   
1458628  id0986544          2  2016-05-30 03:08:19  2016-05-30 03:14:10   
1458629  id3109086          2  2016-06-24 10:33:51  2016-06-24 10:43:52   
1458630  id0287353          2  2016-06-25 03:44:32  2016-06-25 03:53:41   
1458631  id1724231          1  2016-05-14 23:18:23  2016-05-14 23:24:05   
1458632  id0469946          2  2016-03-06 11:04:48  2016-03-06 11:17:45   
1458633  id2432342          1  2016-03-17 19:10:16  2016-03-17 19:26:35   
1458634  id3445276          1  2016-04-03 13:51:25  2016-04-03 14:07:37   
1458635  id3027038          2  2016-05-19